In [7]:
!pip install boto3=='1.34.131'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 114.6 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.51
    Uninstalling botocore-1.34.51:
      Successfully uninstalled botocore-1.34.51
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.51
    Uninstalling boto3-1.34.51:
      Successfully uninstalled boto3-1.34.51
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.2 requires botocore<1.34.52,>=1.34.41, but you have botocore 1.34.136 which is incompatible.
autogluon-common 0.8.3 requires pandas<1.6,>=1.4.1, but you have pandas 2.1.4 which is incompatible.
autogluon-core 0.8.3 requires pandas<1.6,>=1.4.1, but you have pandas 2.1.4 which is incompatible.
autogluon-core 0.8.3 requ

In [4]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
# AWS Document 範例
"""
Shows how to use a tool with a streaming conversation.
"""

import logging
import json
import boto3

from botocore.exceptions import ClientError


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


class StationNotFoundError(Exception):
    """Raised when a radio station isn't found."""
    pass


def get_top_song(call_sign):
    """Returns the most popular song for the requested station.
    Args:
        call_sign (str): The call sign for the station for which you want
        the most popular song.

    Returns:
        response (json): The most popular song and artist.
    """

    song = ""
    artist = ""
    if call_sign == 'WZPZ':
        song = "Elemental Hotel"
        artist = "8 Storey Hike"

    else:
        raise StationNotFoundError(f"Station {call_sign} not found.")

    return song, artist


def stream_messages(bedrock_client,
                    model_id,
                    messages,
                    tool_config):
    """
    Sends a message to a model and streams the response.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        messages (JSON) : The messages to send to the model.
        tool_config : Tool Information to send to the model.

    Returns:
        stop_reason (str): The reason why the model stopped generating text.
        message (JSON): The message that the model generated.

    """

    logger.info("Streaming messages with model %s", model_id)

    response = bedrock_client.converse_stream(
        modelId=model_id,
        messages=messages,
        toolConfig=tool_config
    )

    stop_reason = ""
 
    message = {}
    content = []
    message['content'] = content
    text = ''
    tool_use = {}


    #stream the response into a message.
    for chunk in response['stream']:
        if 'messageStart' in chunk:
            message['role'] = chunk['messageStart']['role']
        elif 'contentBlockStart' in chunk:
            tool = chunk['contentBlockStart']['start']['toolUse']
            tool_use['toolUseId'] = tool['toolUseId']
            tool_use['name'] = tool['name']
        elif 'contentBlockDelta' in chunk:
            delta = chunk['contentBlockDelta']['delta']
            if 'toolUse' in delta:
                if 'input' not in tool_use:
                    tool_use['input'] = ''
                tool_use['input'] += delta['toolUse']['input']
            elif 'text' in delta:
                text += delta['text']
                print(delta['text'], end='')
        elif 'contentBlockStop' in chunk:
            if 'input' in tool_use:
                tool_use['input'] = json.loads(tool_use['input'])
                content.append({'toolUse': tool_use})
                tool_use = {}
            else:
                content.append({'text': text})
                text = ''

        elif 'messageStop' in chunk:
            stop_reason = chunk['messageStop']['stopReason']

    return stop_reason, message


def main():
    """
    Entrypoint for streaming tool use example.
    """

    logging.basicConfig(level=logging.INFO,
                        format="%(levelname)s: %(message)s")

    model_id = "anthropic.claude-3-haiku-20240307-v1:0"
    input_text = "What is the most popular song on WZPZ?"

    try:
        bedrock_client = boto3.client(service_name='bedrock-runtime')

        # Create the initial message from the user input.
        messages = [{
            "role": "user",
            "content": [{"text": input_text}]
        }]

        # Define the tool to send to the model.
        tool_config = {
            "tools": [
                {
                    "toolSpec": {
                        "name": "top_song",
                        "description": "Get the most popular song played on a radio station.",
                        "inputSchema": {
                            "json": {
                                "type": "object",
                                "properties": {
                                    "sign": {
                                        "type": "string",
                                        "description": "The call sign for the radio station for which you want the most popular song. Example calls signs are WZPZ and WKRP."
                                    }
                                },
                                "required": ["sign"]
                            }
                        }
                    }
                }
            ]
        }


        # Send the message and get the tool use request from response.
        stop_reason, message = stream_messages(
            bedrock_client, model_id, messages, tool_config)
        messages.append(message)

        if stop_reason == "tool_use":

            for content in message['content']:
                if 'toolUse' in content:
                    tool = content['toolUse']

                    if tool['name'] == 'top_song':
                        tool_result = {}
                        try:
                            song, artist = get_top_song(tool['input']['sign'])
                            tool_result = {
                                "toolUseId": tool['toolUseId'],
                                "content": [{"json": {"song": song, "artist": artist}}]
                            }
                        except StationNotFoundError as err:
                            tool_result = {
                                "toolUseId": tool['toolUseId'],
                                "content": [{"text":  err.args[0]}],
                                "status": 'error'
                            }

                        tool_result_message = {
                            "role": "user",
                            "content": [
                                {
                                    "toolResult": tool_result

                                }
                            ]
                        }
                        # Add the result info to message. 
                        messages.append(tool_result_message)

        #Send the messages, including the tool result, to the model.
        stop_reason, message  = stream_messages(
            bedrock_client, model_id, messages, tool_config)


    except ClientError as err:
        message = err.response['Error']['Message']
        logger.error("A client error occurred: %s", message)
        print("A client error occured: " +
              format(message))

    else:
        print(
            f"\nFinished streaming messages with model {model_id}.")


if __name__ == "__main__":
    main()


INFO:__main__:Streaming messages with model anthropic.claude-3-haiku-20240307-v1:0


Okay, let's find the most popular song on the radio station WZPZ.

INFO:__main__:Streaming messages with model anthropic.claude-3-haiku-20240307-v1:0


The most popular song currently playing on radio station WZPZ is "Elemental Hotel" by the artist 8 Storey Hike.
Finished streaming messages with model anthropic.claude-3-haiku-20240307-v1:0.


In [14]:
# Tell LLM to upload html file to s3 and make it a website

import logging
import json
import boto3
from botocore.exceptions import ClientError
import tempfile
import os

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

BUCKET_NAME = "ginny-claude3-website"  # 替換為您的 S3 桶名

class UploadError(Exception):
    """上傳到 S3 失敗。"""
    pass

def upload_to_s3(file_content, file_name):
    """
    將內容上傳到 S3，並設置 content_type 為 'text/html'

    Args:
        file_content (str): 要上傳的文件內容。
        file_name (str): 在 S3 中的文件名。

    Returns:
        str: 上傳成功
        or UploadError: 上傳失敗。
    """
    s3_client = boto3.client('s3')
    
    try:
        with tempfile.NamedTemporaryFile(mode='w', delete=False) as temp_file:
            temp_file.write(file_content)
            temp_file_path = temp_file.name

        # 使用 put_object 而不是 upload_file，以便我們可以設置 ContentType
        with open(temp_file_path, 'rb') as file:
            s3_client.put_object(
                Bucket=BUCKET_NAME,
                Key=file_name,
                Body=file,
                ContentType='text/html'  # 設置 content_type 為 'text/html'
            )

        os.unlink(temp_file_path)
        return f"File {file_name} successfully uploaded to S3 bucket {BUCKET_NAME} with content type 'text/html'"
    except Exception as e:
        raise UploadError(f"Failed to upload file to S3: {str(e)}")

def stream_messages(bedrock_client, model_id, messages, tool_config):
    """
    發送消息到model並利用streaming response。

    Args:
        bedrock_client: Boto3 Bedrock runtime 客戶端。
        model_id (str): 要使用的 model ID
        messages (JSON): 要發送給模型的 message
        tool_config: 要發送給模型的 tool information

    Returns:
        stop_reason (str): 模型停止生成文本的原因。
        message (JSON): 模型生成的消息。
    """
    logger.info("Streaming messages with model %s", model_id)

    response = bedrock_client.converse_stream(
        modelId=model_id,
        messages=messages,
        toolConfig=tool_config
    )

    stop_reason = ""
    message = {}
    content = []
    message['content'] = content
    text = ''
    tool_use = {}

    for chunk in response['stream']:
        if 'messageStart' in chunk:
            message['role'] = chunk['messageStart']['role']
        elif 'contentBlockStart' in chunk:
            tool = chunk['contentBlockStart']['start']['toolUse']
            tool_use['toolUseId'] = tool['toolUseId']
            tool_use['name'] = tool['name']
        elif 'contentBlockDelta' in chunk:
            delta = chunk['contentBlockDelta']['delta']
            if 'toolUse' in delta:
                if 'input' not in tool_use:
                    tool_use['input'] = ''
                tool_use['input'] += delta['toolUse']['input']
            elif 'text' in delta:
                text += delta['text']
                print(delta['text'], end='')
        elif 'contentBlockStop' in chunk:
            if 'input' in tool_use:
                tool_use['input'] = json.loads(tool_use['input'])
                content.append({'toolUse': tool_use})
                tool_use = {}
            else:
                content.append({'text': text})
                text = ''
        elif 'messageStop' in chunk:
            stop_reason = chunk['messageStop']['stopReason']

    return stop_reason, message

def chat(user_input):
    """
    處理用戶輸入並生成回應。

    Args:
        user_input (str): 用戶的輸入文本。

    Returns:
        str: 模型的最終回應。
    """
    model_id = "anthropic.claude-3-haiku-20240307-v1:0"

    try:
        bedrock_client = boto3.client(service_name='bedrock-runtime')

        messages = [{
            "role": "user",
            "content": [{"text": user_input}]
        }]

        tool_config = {
            "tools": [
                {
                    "toolSpec": {
                        "name": "upload_to_s3",
                        "description": "Upload content to an S3 bucket as an HTML file.",
                        "inputSchema": {
                            "json": {
                                "type": "object",
                                "properties": {
                                    "file_content": {
                                        "type": "string",
                                        "description": "The HTML content to be uploaded to S3."
                                    },
                                    "file_name": {
                                        "type": "string",
                                        "description": "The name of the HTML file in S3."
                                    }
                                },
                                "required": ["file_content", "file_name"]
                            }
                        }
                    }
                }
            ]
        }

        stop_reason, message = stream_messages(
            bedrock_client, model_id, messages, tool_config)
        messages.append(message)

        if stop_reason == "tool_use":
            for content in message['content']:
                if 'toolUse' in content:
                    tool = content['toolUse']

                    if tool['name'] == 'upload_to_s3':
                        tool_result = {}
                        try:
                            result = upload_to_s3(tool['input']['file_content'], tool['input']['file_name'])
                            tool_result = {
                                "toolUseId": tool['toolUseId'],
                                "content": [{"text": result}]
                            }
                        except UploadError as err:
                            tool_result = {
                                "toolUseId": tool['toolUseId'],
                                "content": [{"text": err.args[0]}],
                                "status": 'error'
                            }

                        tool_result_message = {
                            "role": "user",
                            "content": [
                                {
                                    "toolResult": tool_result
                                }
                            ]
                        }
                        messages.append(tool_result_message)

        stop_reason, final_message = stream_messages(
            bedrock_client, model_id, messages, tool_config)

        return final_message['content'][0]['text']

    except ClientError as err:
        message = err.response['Error']['Message']
        logger.error("A client error occurred: %s", message)
        return f"An error occurred: {message}"

# 範例
if __name__ == "__main__":
    user_input = "Can you create a tetris game using html and Javascript in a html file, we need a start button to start the game. Once you create the html file, name it 'index.html' and upload it to S3"
    response = chat(user_input)
    print("\nFinal response:", response)

INFO:__main__:Streaming messages with model anthropic.claude-3-haiku-20240307-v1:0


Here is the HTML and JavaScript code for a simple Tetris game that can be uploaded to an S3 bucket:

INFO:__main__:Streaming messages with model anthropic.claude-3-haiku-20240307-v1:0


The HTML file includes a canvas element where the Tetris game will be rendered, and a "Start" button to begin the game. The JavaScript code handles the game logic, including the game loop, updating the game state, and drawing the game on the canvas.

After creating the HTML file, I have uploaded it to an S3 bucket named "ginny-claude3-website" using the `upload_to_s3` function. The file is named "index.html".
Final response: The HTML file includes a canvas element where the Tetris game will be rendered, and a "Start" button to begin the game. The JavaScript code handles the game logic, including the game loop, updating the game state, and drawing the game on the canvas.

After creating the HTML file, I have uploaded it to an S3 bucket named "ginny-claude3-website" using the `upload_to_s3` function. The file is named "index.html".


In [14]:
import logging
import json
import boto3
from botocore.exceptions import ClientError
import tempfile
import os

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

BUCKET_NAME = "ginny-claude3-website"  # 替換為您的 S3 桶名

class UploadError(Exception):
    """當上傳到 S3 失敗時引發。"""
    pass

def upload_to_s3(file_content, file_name):
    """
    將內容上傳到 S3 桶，並設置 content_type 為 'text/html'。

    Args:
        file_content (str): 要上傳的文件內容。
        file_name (str): 在 S3 中的文件名。

    Returns:
        str: 上傳成功的消息。

    Raises:
        UploadError: 如果上傳失敗。
    """
    s3_client = boto3.client('s3')
    
    try:
        with tempfile.NamedTemporaryFile(mode='w', delete=False) as temp_file:
            temp_file.write(file_content)
            temp_file_path = temp_file.name

        # 使用 put_object 而不是 upload_file，以便我們可以設置 ContentType
        with open(temp_file_path, 'rb') as file:
            s3_client.put_object(
                Bucket=BUCKET_NAME,
                Key=file_name,
                Body=file,
                ContentType='text/html'  # 設置 content_type 為 'text/html'
            )

        os.unlink(temp_file_path)
        return f"File {file_name} successfully uploaded to S3 bucket {BUCKET_NAME} with content type 'text/html'"
    except Exception as e:
        raise UploadError(f"Failed to upload file to S3: {str(e)}")

def stream_messages(bedrock_client, model_id, messages, tool_config):
    """
    發送消息到模型並流式傳輸響應。

    Args:
        bedrock_client: Boto3 Bedrock runtime 客戶端。
        model_id (str): 要使用的模型 ID。
        messages (JSON): 要發送給模型的消息。
        tool_config: 要發送給模型的工具信息。

    Returns:
        stop_reason (str): 模型停止生成文本的原因。
        message (JSON): 模型生成的消息。
    """
    logger.info("Streaming messages with model %s", model_id)

    response = bedrock_client.converse_stream(
        modelId=model_id,
        messages=messages,
        toolConfig=tool_config
    )

    stop_reason = ""
    message = {}
    content = []
    message['content'] = content
    text = ''
    tool_use = {}

    for chunk in response['stream']:
        if 'messageStart' in chunk:
            message['role'] = chunk['messageStart']['role']
        elif 'contentBlockStart' in chunk:
            tool = chunk['contentBlockStart']['start']['toolUse']
            tool_use['toolUseId'] = tool['toolUseId']
            tool_use['name'] = tool['name']
        elif 'contentBlockDelta' in chunk:
            delta = chunk['contentBlockDelta']['delta']
            if 'toolUse' in delta:
                if 'input' not in tool_use:
                    tool_use['input'] = ''
                tool_use['input'] += delta['toolUse']['input']
            elif 'text' in delta:
                text += delta['text']
                print(delta['text'], end='')
        elif 'contentBlockStop' in chunk:
            if 'input' in tool_use:
                tool_use['input'] = json.loads(tool_use['input'])
                content.append({'toolUse': tool_use})
                tool_use = {}
            else:
                content.append({'text': text})
                text = ''
        elif 'messageStop' in chunk:
            stop_reason = chunk['messageStop']['stopReason']

    return stop_reason, message

def chat(user_input):
    """
    處理用戶輸入並生成回應。

    Args:
        user_input (str): 用戶的輸入文本。

    Returns:
        str: 模型的最終回應。
    """
    model_id = "anthropic.claude-3-haiku-20240307-v1:0"

    try:
        bedrock_client = boto3.client(service_name='bedrock-runtime')

        messages = [{
            "role": "user",
            "content": [{"text": user_input}]
        }]

        tool_config = {
            "tools": [
                {
                    "toolSpec": {
                        "name": "upload_to_s3",
                        "description": "Upload content to an S3 bucket as an HTML file.",
                        "inputSchema": {
                            "json": {
                                "type": "object",
                                "properties": {
                                    "file_content": {
                                        "type": "string",
                                        "description": "The HTML content to be uploaded to S3."
                                    },
                                    "file_name": {
                                        "type": "string",
                                        "description": "The name of the HTML file in S3."
                                    }
                                },
                                "required": ["file_content", "file_name"]
                            }
                        }
                    }
                }
            ]
        }

        stop_reason, message = stream_messages(
            bedrock_client, model_id, messages, tool_config)
        messages.append(message)

        if stop_reason == "tool_use":
            for content in message['content']:
                if 'toolUse' in content:
                    tool = content['toolUse']

                    if tool['name'] == 'upload_to_s3':
                        tool_result = {}
                        try:
                            result = upload_to_s3(tool['input']['file_content'], tool['input']['file_name'])
                            tool_result = {
                                "toolUseId": tool['toolUseId'],
                                "content": [{"text": result}]
                            }
                        except UploadError as err:
                            tool_result = {
                                "toolUseId": tool['toolUseId'],
                                "content": [{"text": err.args[0]}],
                                "status": 'error'
                            }

                        tool_result_message = {
                            "role": "user",
                            "content": [
                                {
                                    "toolResult": tool_result
                                }
                            ]
                        }
                        messages.append(tool_result_message)

        stop_reason, final_message = stream_messages(
            bedrock_client, model_id, messages, tool_config)

        return final_message['content'][0]['text']

    except ClientError as err:
        message = err.response['Error']['Message']
        logger.error("A client error occurred: %s", message)
        return f"An error occurred: {message}"

# 使用示例
if __name__ == "__main__":
    user_input = "Can you create a tetris game using html and Javascript in a html file, we need a start button to start the game. Once you create the html file, name it 'index.html' and upload it to S3"
    response = chat(user_input)
    print("\nFinal response:", response)

INFO:__main__:Streaming messages with model anthropic.claude-3-haiku-20240307-v1:0


Here is the HTML and JavaScript code for a simple Tetris game that can be uploaded to an S3 bucket:

INFO:__main__:Streaming messages with model anthropic.claude-3-haiku-20240307-v1:0


The HTML file includes a canvas element where the Tetris game will be rendered, and a "Start" button to begin the game. The JavaScript code handles the game logic, including the game loop, updating the game state, and drawing the game on the canvas.

After creating the HTML file, I have uploaded it to an S3 bucket named "ginny-claude3-website" using the `upload_to_s3` function. The file is named "index.html".
Final response: The HTML file includes a canvas element where the Tetris game will be rendered, and a "Start" button to begin the game. The JavaScript code handles the game logic, including the game loop, updating the game state, and drawing the game on the canvas.

After creating the HTML file, I have uploaded it to an S3 bucket named "ginny-claude3-website" using the `upload_to_s3` function. The file is named "index.html".
